In [1]:
# Set Variables
SW_LAT = -38.88917
SW_LON = 140.84444
NE_LAT = -33.70833
NE_LON = 150.0
TIMESCALE = "Monthly"

PARAMETER_NAME = f"Pat6_C_B_1_{TIMESCALE}Mean"
PROPERTY_NAME = "Storage Volume"
TIME_PERIOD = "2010/2024"


In [2]:
import requests
from xml.etree.ElementTree import Element, SubElement, tostring

# Define the namespace map to use in the XML
ns = {
    'sos': "http://www.opengis.net/sos/2.0",
    'fes': "http://www.opengis.net/fes/2.0",
    'gml': "http://www.opengis.net/gml/3.2",
    'xsi': "http://www.w3.org/2001/XMLSchema-instance",
    'sams': "http://www.opengis.net/samplingSpatial/2.0"
}

# Define the endpoint URL for the SOS service
url = "http://www.bom.gov.au/waterdata/services"

# Create XML elements for the request
envelope = Element('sos:GetFeatureOfInterest', {
    'service': 'SOS', 'version': '2.0', 'xmlns:sos': ns['sos'], 'xmlns:fes': ns['fes'], 'xmlns:gml': ns['gml'], 'xmlns:xsi': ns['xsi'], 'xmlns:sams': ns['sams']
})

spatial_filter = SubElement(envelope, 'sos:spatialFilter')
bbox = SubElement(spatial_filter, 'fes:BBOX')
value_ref = SubElement(bbox, 'fes:ValueReference')
value_ref.text = 'om:featureOfInterest/*/sams:shape'

envelope_ = SubElement(bbox, 'gml:Envelope', {
    'srsName': 'urn:ogc:def:crs:EPSG::4326'
})
lower_corner = SubElement(envelope_, 'gml:lowerCorner')
upper_corner = SubElement(envelope_, 'gml:upperCorner')

# Insert the actual coordinates for the Murray River bounding box
lower_corner.text = f"{SW_LAT} {SW_LON}"  # Southernmost, westernmost
upper_corner.text = f"{NE_LAT} {NE_LON}"  # Northernmost, easternmost

# Add observed property element
observed_property = SubElement(envelope, 'sos:observedProperty')
observed_property.text = f"http://bom.gov.au/waterdata/services/parameters/{PROPERTY_NAME}"  # Example observed property


# Convert the ElementTree to a string
request_xml = tostring(envelope, 'utf-8')

# Define headers for the POST request
headers = {
    'Content-Type': 'application/xml'
}

# Send the POST request
FeatureOfInterestResponse = requests.post(url, data=request_xml, headers=headers)

# Print the response from the server


In [3]:
import requests
import xml.etree.ElementTree as ET
# Sample XML response from GetFeatureOfInterest
xml_response = FeatureOfInterestResponse.text

# Parse the XML string
root = ET.fromstring(xml_response)

# Extract station IDs
station_ids = [elem.text.split('/')[-1] for elem in root.findall('.//gml:identifier', namespaces={'gml': 'http://www.opengis.net/gml/3.2'})]

print(station_ids) 

#get stations within region


['408216A', 'RE931', 'SB898', '41010307', '228268A', '41010904', '41010293', '228265A', '410102', '219027', '407327A', '410190', '410131', 'BARKERSCREEK', '410717', 'RE670', 'BUANGOR_STORAGE', '228263A', '407335A', '219033', '41010290', 'sp-o10138', 'SB855', 'SB855B', '410742', '410704', '228269A', '401224A', '228264A', 'sp-o10298', 'DOCK', 'sp-o10334', '41010297', 'ELMHURST', '233272A', '228267A', '41010294', '41010295', '41010922', '41010300', '229407A', '401570', '410748', 'sp-o10438', 'GREEN', 'sp-o10454', '410788', '222537', '41010928', '409011', '401027', '41010292', '222539', '230216A', '410573', '231222A', 'RES_BEEWKFS', 'RE690', '226236A', '232219A', '232220A', '232222A', '415275A', '407326A', 'sp-o10606', 'BATYO_CATYO', 'LAKE_BELLFIELD', 'sp-o10109', 'sp-o10350', '222538', 'LAKE_FYANS', '222540', 'LAKE_LONSDALE', 'sp-o10806', 'sp-o10930', '4261093', 'A4261093', 'A4260553', 'sp-o11590', 'LAURISTON', 'RES_BENWLOR', '225225A', '41010941', '407333A', '41010304', '229409A', '41010

In [4]:
import requests
from xml.etree import ElementTree as ET
import pandas as pd

# Define the SOS2 endpoint URL
url = "http://www.bom.gov.au/waterdata/services"

data = []  # List to store data for DataFrame


# Define the parameters for the GetObservation request
params = {
    'service': 'SOS',
    'version': '2.0.0',
    'request': 'GetObservation',
    'procedure': f"http://bom.gov.au/waterdata/services/tstypes/{PARAMETER_NAME}",
    'observedProperty': f"http://bom.gov.au/waterdata/services/parameters/{PROPERTY_NAME}",
    'temporalFilter': f"om:phenomenonTime,{TIME_PERIOD}"
}

# Assuming station_ids is defined earlier in the script
for station_id in station_ids:
    print(f"Querying data for station: {station_id}")
    
    # Add the station ID to the parameters
    params['featureOfInterest'] = f"http://bom.gov.au/waterdata/services/stations/{station_id}"
    
    # Make the GetObservation request to the SOS2 endpoint
    response = requests.get(url, params=params, timeout=100)  # Added timeout
    
    if response.status_code == 200:
        # Parse the XML response
        root = ET.fromstring(response.content)
        
        # Find all the MeasurementTVP elements which contain the time-value pairs
        tvp_elements = root.findall('.//{http://www.opengis.net/waterml/2.0}MeasurementTVP')
        
        if not tvp_elements:
            print("No data found for this station.")
        else:
            # Print out the time and value for each MeasurementTVP
            for tvp in tvp_elements:
                time = tvp.find('{http://www.opengis.net/waterml/2.0}time').text
                value = tvp.find('{http://www.opengis.net/waterml/2.0}value').text
                #print(f"  {time}: {value}")
                data.append({'Station': station_id, 'Time': time, 'Value': value})
    else:
        print(f"Failed to retrieve data for station {station_id}. HTTP status code: {response.status_code}")
        print("Response Headers:", response.headers)
        print("Response Body:", response.text)


df = pd.DataFrame(data)

pivot_df = df.pivot_table(index='Time', values='Value', columns='Station', aggfunc='first')

# Show pivoted DataFrame
print(pivot_df)
# Show DataFrame

Querying data for station: 408216A
Querying data for station: RE931
Querying data for station: SB898
Querying data for station: 41010307
No data found for this station.
Querying data for station: 228268A
Querying data for station: 41010904
Querying data for station: 41010293
No data found for this station.
Querying data for station: 228265A
Querying data for station: 410102
Querying data for station: 219027
Querying data for station: 407327A
Querying data for station: 410190
Querying data for station: 410131
Querying data for station: BARKERSCREEK
Querying data for station: 410717
Querying data for station: RE670
Querying data for station: BUANGOR_STORAGE
Querying data for station: 228263A
Querying data for station: 407335A
Querying data for station: 219033
Querying data for station: 41010290
No data found for this station.
Querying data for station: sp-o10138
Querying data for station: SB855
Querying data for station: SB855B
Querying data for station: 410742
Querying data for station:

In [5]:
pivot_df.to_csv(f'{TIMESCALE}MeanStorageVolume.csv', index=True)

